# Gatsiva Sample Analysis
## Visualizing Return Results

This notebook shows how to visualize the distribution of returns after an event occurs utilizing the Gatsiva API.

API access is **currently limited to beta testers and collaborators**. For more information on how to utilize the Gatsiva API or to request access and an API key, please visit the [Gatsiva Website](https://gatsiva.com). For more information on the Gatsiva API, please visit the [Gatsiva API Documentation](https://gatsiva.com/docs).

## Enter Input Information Here

In [1]:
symbol = 'BTC:USD:hourly'
condition = 'close(1) < low(168) 1 period ago'
stddev_bands = 1.5
periods = 336
usingdatafrom = "2016-12-04"
usingdatato = "2018-01-10"
apikey = "<your_api_key>"

#### Python code to get the results - see results further below

In [2]:
# Install Prerequesities if needed
#
# Run this (uncomment below) to install / upgrade Plotly
#!pip install plotly --upgrade


In [3]:
# Import utility libraries we will need
import requests
import json
import numpy as np
import pandas as pd
import warnings

# Import plotly for graphing awesomeness
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

# To allow the jupyter notebook to render in offline mode
init_notebook_mode(connected=True)

# Ignore warnings
warnings.filterwarnings('ignore')

# Form the request
request_data = {'symbol':symbol,'condition_id':1,'condition':condition,"periods":periods,'from':usingdatafrom,'to':usingdatato}
bearer_token = 'Bearer ' + apikey
my_headers = {'Accept': 'application/json', 'Authorization': bearer_token}

# Send the request
analytic = requests.post('https://api.gatsiva.com/api/v1/conditions/return_profile',json=request_data,headers=my_headers)

# Pull out the results and format it into a pandas data frame
results = analytic.json()['results']
df = pd.read_json(json.dumps(results), orient='records')

# Now add two new columns to support lines for 2x stddevs
df['upper'] = df['mean'] + ((stddev_bands/2) * df['stddev'])
df['lower'] = df['mean'] - ((stddev_bands/2) * df['stddev'])

# Get the number of observations
num_observations = results[0]['n']

# Create the data structure needed for plotly
data = [
    Scatter(
        x=df['returnperiod'],
        y=df['mean'],
        name='mean'
    ),
    Scatter(
        x=df['returnperiod'], 
        y=df['upper'],
        name=str(stddev_bands) + ' stddev band upper'
    ),
    Scatter(
        x=df['returnperiod'], 
        y=df['lower'],
        name=str(stddev_bands) + ' stddev band lower'

    )
]

# Create the layout needed for plotly
layout = Layout(
    title=condition + ' -- ' + str(num_observations) + ' observations',
    yaxis=dict(title='return'),
    xaxis=dict(title='periods after event'),
    legend=dict(orientation="h",x=0.15, y=-0.2)
)

# Create the figure
fig = Figure(data=data, layout=layout)

## Results

In [4]:
# Plot the result
iplot(fig)

### Detailed Results

Below here, you can see the detailed results within the dataframe. Note how the upper and lower columns were added in the earlier step.

In [5]:
df

,max,mean,median,min,n,p25th,p75th,returnperiod,stddev,upper,lower
0,0.113561,0.006080,0.006018,-0.037435,75,-0.008651,0.017632,1,0.025642,0.025312,-0.013151
1,0.126917,0.011294,0.008238,-0.064758,75,-0.005965,0.028485,2,0.033837,0.036672,-0.014084
2,0.212311,0.017107,0.013291,-0.074445,75,-0.005334,0.034466,3,0.042320,0.048847,-0.014633
3,0.235700,0.018028,0.010852,-0.091865,75,-0.010383,0.039997,4,0.049405,0.055082,-0.019026
4,0.229625,0.019390,0.012953,-0.093329,75,-0.010978,0.035616,5,0.055442,0.060971,-0.022191
5,0.219393,0.019854,0.013184,-0.096483,75,-0.009037,0.040164,6,0.054734,0.060905,-0.021196
6,0.191785,0.018621,0.009701,-0.111560,75,-0.021104,0.042648,7,0.059437,0.063199,-0.025957
7,0.221436,0.020853,0.008483,-0.084777,75,-0.018598,0.045040,8,0.062735,0.067904,-0.026198
8,0.232986,0.020769,0.004569,-0.098534,75,-0.024504,0.042968,9,0.068038,0.071798,-0.030260
9,0.262723,0.019495,0.008197,-0.091449,75,-0.028630,0.047655,10,0.068832,0.071119,-0.032129
